In [122]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import os
import re
import xlwt

input1 = input('1. 유튜브에서 검색할 주제 키워드를 입력하세요(예:롯데마트) ')
#input1 = '카리나'
#input2 = int(input('2. 위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?: '))
input2 = 2
# input3 = int(input('3. 각 동영상에서 추출할 댓글은 몇건입니까?: '))
input3 = 2
# f_name = input('5. 크롤링 결과를 저장할 폴더명만 쓰세요:')
f_dir = 'C:\\py_temp\\test\\'

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+input1)
os.chdir(f_dir+s+'-'+input1)

ff_name=f_dir+s+'-'+input1+'\\'+s+'-'+input1+'.txt'
fc_name=f_dir+s+'-'+input1+'\\'+s+'-'+input1+'.csv'
fx_name=f_dir+s+'-'+input1+'\\'+s+'-'+input1+'.xls'

s_time = time.time( )

if os.path.isdir(f_dir):
    print('입력하신 경로가 존재하여 {} 폴더에 저장하겠습니다'.format(f_dir))
else:
    os.makedirs(f_dir)
    print('입력하신 경로가 존재하지 않아 {} 폴더를 생성했습니다'.format(f_dir))

s_time = time.time()

options = webdriver.ChromeOptions()
# 웹 드라이버 실행
driver_path = r"C:\py_temp\chromedriver\chromedriver.exe"
driver = webdriver.Chrome(driver_path)

def auto_scroll(driver):
    driver.execute_script("window.scrollTo(0, 3000)")
    time.sleep(5)  # 5초 대기

# 유튜브 접속
driver.get("https://www.youtube.com")
time.sleep(2)
commentCnt =0
cmtCnt = []
videoCnt = 0
num = []
url = []
nickname = []
commentDate = []
commentContent = []
tmp_commentDate = ""
cont = ""

# 검색어 입력
element = driver.find_element(By.CSS_SELECTOR, "input#search")
element.send_keys(input1)
element.send_keys(Keys.ENTER)

full_html = driver.page_source
soup = BeautifulSoup(full_html, 'html.parser')

href_list = []


time.sleep(2)

judgment = 0

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Find all 'a' elements with id="thumbnail"
a_elements = soup.find_all('a', class_="yt-simple-endpoint style-scope ytd-video-renderer")
href_list = [a['href'] for a in a_elements]
href_list = [href for href in href_list if '/shorts' not in href]

videoCnt = 0

# 비디오 검색 결과 수집
for link in href_list:
    if videoCnt == input2:
        break;
    time.sleep(2)

    new_link = "https://www.youtube.com" + link

    driver.get(new_link)
    
    time.sleep(1)

    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
    #driver.execute_script("window.scrollTo(0, 100)")
    time.sleep(1)
    
    auto_scroll(driver)

    time.sleep(1)

    print('주소:', new_link)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    video_list = soup.find_all('ytd-comment-thread-renderer', 'style-scope ytd-item-section-renderer')
    
    videoCnt += 1 
    for j in video_list:  # 영상 개수     
#         if driver.current_url != new_link: 
#             videoCnt += 1 
        num.append(videoCnt)
        print('영상 수:', videoCnt)  
        
        tmp_url = new_link
        url.append(tmp_url)
        print('url:', tmp_url) 
                       
        tmp_nickname = j.find('h3', class_='style-scope ytd-comment-renderer').get_text()
        nickname.append(tmp_nickname.strip())
        print('댓글 작성자명:', tmp_nickname)

        tmp_commentDate = j.find('a', 'yt-simple-endpoint style-scope yt-formatted-string').get_text()
        commentDate.append(tmp_commentDate.strip())
        print('작성일자:', tmp_commentDate)
        
        commentCnt +=1
        cmtCnt.append(commentCnt)
        
        comment = j.find('yt-formatted-string', class_='style-scope ytd-comment-renderer').get_text()
        commentContent.append(comment.strip())
        print('댓글 내용:', comment)
        if commentCnt >= input3:
            break
    
    
       
        
nickname = nickname[:len(url)]
commentDate = commentDate[:len(url)]
commentContent = commentContent[:len(url)]      

result = pd.DataFrame()
result['번호'] = num
result['URL 주소'] = url
result['댓글작성자명'] = nickname
result['댓글 작성일자'] = commentDate
result['댓글 내용'] = commentContent


with open(ff_name, 'a', encoding='UTF-8') as f:
    for i in range(videoCnt):
        f.write(str(num[i]) + '번째 영상의 ' + str(cmtCnt[i]) + '번째 댓글 추출\n')
        f.write('1. URL 주소:' + url[i].strip() + '\n')
        f.write('2. 댓글 작성자명:' + nickname[i].strip() + '\n')
        f.write('3. 댓글 작성일자:' + commentDate[i].strip() + '\n')
        f.write('4. 댓글 내용:' + commentContent[i].strip() + '\n')
        f.write('\n')
f.close()

# 엑셀 파일 저장
result.to_excel(fx_name , index=False)

# csv 형태로 저장하기
result.to_csv(fc_name,encoding="utf-8-sig",index=False)

driver.close()


입력하신 경로가 존재하여 C:\py_temp\test\ 폴더에 저장하겠습니다
주소: https://www.youtube.com/watch?v=JTqCtl548oc&pp=ygUJ7Lm066as64KY
영상 수: 1
url: https://www.youtube.com/watch?v=JTqCtl548oc&pp=ygUJ7Lm066as64KY
댓글 작성자명: 


              차린건 쥐뿔도 없지만
            


작성일자: 2주 전(수정됨)
댓글 내용: 안녕하세요 <차린건 쥐뿔도 없지만> 제작진입니다. 업로드 이후 발견된 음원 저작권 문제로 일부 구간 (12:18초, 26:13초) 이 삭제되었습니다. 

수정된 영상의 싱크가 유튜브 서버 내에서 순차적으로 적용되고 있어 TV 수상기를 통한 시청시 싱크가 안 맞는 현상이 일부 발생하고 있는 것으로 확인됩니다. 유튜브 내 서버 최적화 기간 동안 모바일 시청을 권장드리며 해당 구간은 추후 비하인드 영상에 추가할 예정입니다.  

시청에 불편을 드려 다시 한 번 죄송합니다.

Hello, This is the team of "Nothing prepared". Two sections of this episode (12:18, 26:13) have been deleted due to a issue of background music copyright which have been found after the upload.

As the sync of the modified video is being applied gradually within the YouTube server, we have been noticed that sync errors are occurring when you are watching through the Television devices. According to that, we are recommending to watch it through mobile or pc during the